### Test case LVV-T60 - Verify implementation of Publishing predicted visit schedule
Verify that a predict-visit schedule can be published by the OCS. <br>
DMS-REQ-0353-V-01: Publishing predicted visit schedule

### Discussion 
Description: Use simulated schedule and test that an external unauthenticated user can retrieve the information.

**Specification:** A service shall be provided to publish to the community the next visit location and the predicted visit schedule provided by the OCS. This service shall consist of both a web page for human inspection and a web API to allow automated tools to respond promptly.

**Discussion:** The next visit and advanced schedule do not need to be published using the same service or protocol.

In [ ]:
import os
import requests
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Inspect the service

In [ ]:
# URL for the service
obsloctap_url = "https://usdf-rsp-dev.slac.stanford.edu/obsloctap"

In [ ]:
response = requests.get(obsloctap_url)
assert response.status_code == 200, f"request failed with status {response.status_code}"

The service web page is accessible programatically as a web API. Below is a snapshot the serivce, showing it is available for human inslection. The current format of the data is raw, but does satisfy the requirement that the publshed schedule be available for human inspection. Work will be done in Operations to make the visually more pleasing

In [ ]:
# Look at the header information
data = response.json()
assert data is not None

In [ ]:
# Extract information
md = data["metadata"]
print(md.keys())
print(f"ObsLocTap service running at USDF for {md.get("description")} with documentation at  url {md.get("documentation_url")}")

As shown above, the service web page is accessible programatically as a web API, extracting metadata from the service, such as the github URL for the code. The service does not require authentication, a necessary requirent public publishing of the schedule.

Below is a snapshot the serivce, showing it is available for human inslection. The current format of the data is raw, but does satisfy the requirement that the publshed schedule be available for human inspection. Work will be done in Operations to make the visually more pleasing 

In [ ]:
img = mpimg.imread('../figures/test_LVV-T60.png')
plt.imshow(img)
plt.axis('off') 
plt.show()

## Retrieve the schedule 

In [ ]:
# URL for schedule forecast
schedule_url = obsloctap_url + "/schedule" 
assert schedule_url == "https://usdf-rsp-dev.slac.stanford.edu/obsloctap/schedule"
response = requests.get(schedule_url)
assert response.url == "https://usdf-rsp-dev.slac.stanford.edu/obsloctap/schedule"
assert response.status_code == 200, f"request failed with status {response.status_code}"

The schedule serivce is accessible programatically

### The previous 24hrs of observations taken

In [ ]:
# Get the  entries over the last 24 hours (currently returns the top 1000) 
# NOT YET COMPETE
schedule_data_24 = response.json()
len(schedule_data_24)

Define parameters for the next x hours of visits

In [ ]:
# Parameter for look forward time (next visit). Units is hours
# not yet implemented
# Look forward 1 hour
params = {
    "time": "1"
}

In [ ]:
response = requests.get(schedule_url, params=params)
assert response.url == "https://usdf-rsp-dev.slac.stanford.edu/obsloctap/schedule?time=1"
assert response.status_code == 200, f"request failed with status {response.status_code}"

In [ ]:
# NOT COMPLETE
next_visits = response.json()
len(next_visits)

In [ ]:
# Inspect the parameters returned
next_visits[0].keys()

In [ ]:
# Convert to pandas for analysis
next_visits_df = pd.DataFrame(next_visits)
next_visits_df